 ## Summarize the text in the given document using LangChain and OpenAI api

#### Prepare Data
First we prepare the data. For this example we create multiple documents from one long one

In [18]:
from langchain import OpenAI, PromptTemplate
from config import OPENAI_API_KEY

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [19]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('../data/summarize.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)
print(texts)


[Document(page_content='Особливість застосування Української класифікації товарів зовнішньоекономічної діяльності\n\n    Якщо в Українській класифікації товарів зовнішньоекономічної діяльності (далі - УКТ ЗЕД), складеній на основі Гармонізованої системи опису та кодування товарів і Комбінованої номенклатури Європейського Союзу, виявлено неточності перекладу з офіційної мови оригіналу українською мовою назв позицій, підпозицій, а також текстів приміток до розділів, груп, позицій, підпозицій та додаткових приміток, то за основу приймаються тексти офіційних видань Гармонізованої системи опису та кодування товарів і Комбінованої номенклатури Європейського Союзу.\n\nОсновні правила інтерпретації УКТ ЗЕД', metadata={'source': '../data/summarize.txt'}), Document(page_content='Основні правила інтерпретації УКТ ЗЕД\n\n    Класифікація товарів в УКТ ЗЕД здійснюється за такими правилами:\n\n    1. Назви розділів, груп і підгруп наводяться лише для зручності користування УКТ ЗЕД; для юридичних ціл

In [26]:

prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY IN UKRAINIAN:"""


### Refine chain (~2.5-3min)

In [32]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Ukrainian\n"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, chain_type="refine", verbose=True, question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": texts}, return_only_outputs=True)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


Особливість застосування Української класифікації товарів зовнішньоекономічної діяльності

    Якщо в Українській класифікації товарів зовнішньоекономічної діяльності (далі - УКТ ЗЕД), складеній на основі Гармонізованої системи опису та кодування товарів і Комбінованої номенклатури Європейського Союзу, виявлено неточності перекладу з офіційної мови оригіналу українською мовою назв позицій, підпозицій, а також текстів приміток до розділів, груп, позицій, підпозицій та додаткових приміток, то за основу приймаються тексти офіційних видань Гармонізованої системи опису та кодування товарів і Комбінованої номенклатури Європейського Союзу.

Основні правила інтерпретації УКТ ЗЕД


CONCISE SUMMARY IN UKRAINIAN:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an

{'output_text': "\n\nУкраїнська класифікація товарів зовнішньоекономічної діяльності (УКТ ЗЕД) базується на Гармонізованій системі опису та кодування товарів і Комбінованій номенклатурі Європейського Союзу, а також на американському об'є"}

### Map reduce chain (~50sec)

In [23]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True, map_prompt=PROMPT, combine_prompt=PROMPT)
chain({"input_documents": texts}, return_only_outputs=True)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


Особливість застосування Української класифікації товарів зовнішньоекономічної діяльності

    Якщо в Українській класифікації товарів зовнішньоекономічної діяльності (далі - УКТ ЗЕД), складеній на основі Гармонізованої системи опису та кодування товарів і Комбінованої номенклатури Європейського Союзу, виявлено неточності перекладу з офіційної мови оригіналу українською мовою назв позицій, підпозицій, а також текстів приміток до розділів, груп, позицій, підпозицій та додаткових приміток, то за основу приймаються тексти офіційних видань Гармонізованої системи опису та кодування товарів і Комбінованої номенклатури Європейського Союзу.

Основні правила інтерпретації УКТ ЗЕД


CONCISE SUMMARY IN UKRAINIAN:
Prompt after formatting:
Write a concise summary of the following:


Основні правила інтерпретації УКТ ЗЕД

    Класифікація товарів в

{'output_text': '\n\nУкраїнська класифікація товарів зовнішньоекономічної діяльності (УКТ ЗЕД) базується на Гармонізованій системі опису та кодування товарів, Амеріканській Соціальній Тестувальній Матеріалів (ASTM), Міжнародній Непатент'}